In [ ]:
!pip install wordcloud
!pip install TextBlob
!pip install chart-studio
!pip install yfinance

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = '500k-chatgpt-tweets-jan-mar-2023:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3117560%2F5373178%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T143319Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D75e142b71ba028789fd539a2a878e27ec0ab21a4e72b4aa3c44521cd90917d9d4a9730cffe528d386b84c1905c4c2cb74c25062fdc0a8fcb398b2a796dccacd7f76f1fe2a83b783d82a539dccfbcb8d8a92c927a02e82e30c14c8ba75885f161b1b520ad4a5482a218863aae589f4ae594372fe1b816e79f6d9e7015fd690087b5831d1856b91a7bf7399bfc09cc040c23ba7d3800e786605a857b2689f6f0a8dda675823f7ce2b61a6bf7d62ff8804b399db2f231989bb72152fb1c0663eb5e50cfd40ca30d390307e5438105918d68b98eb68d9c3b810708eea90dd9b089dc0d1f3fe6a0eb58160ddc2d5d1e24199813312c5ae9e8186b4cf8b931af16a1dc'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 49816658 bytes downloaded
Downloaded and uncompressed: 500k-chatgpt-tweets-jan-mar-2023
Data source import complete.


In [3]:
!pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.8 MB/s eta 0:00:00


In [4]:
import chart_studio
import re
import string
import collections
import ipywidgets
import cufflinks
import nltk.tokenize

import pandas as pd
import datetime
import seaborn as sns
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import nltk
import gensim
import yfinance as yf

from textblob import TextBlob
from google.colab import widgets
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from tqdm.notebook import tqdm
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
tweet_df = pd.read_csv("//kaggle/input/500k-chatgpt-tweets-jan-mar-2023/Twitter Jan Mar.csv").drop_duplicates()
df = tweet_df.copy(deep = True)
print(df.shape)
df.head().T

(500036, 6)


,0,1,2,3,4
date,2023-03-29 22:58:21+00:00,2023-03-29 22:58:18+00:00,2023-03-29 22:57:53+00:00,2023-03-29 22:57:52+00:00,2023-03-29 22:57:26+00:00
id,1641213230730051584,1641213218520481805,1641213115684536323,1641213110915571715,1641213003260633088
content,"Free AI marketing and automation tools, strate...",@MecoleHardman4 Chat GPT says it’s 15. 😂,https://t.co/FjJSprt0te - Chat with any PDF!\n...,"AI muses: ""In the court of life, we must all f...",Most people haven't heard of Chat GPT yet.\nFi...
username,RealProfitPros,AmyLouWho321,yjleon1976,ChatGPT_Thinks,nikocosmonaut
like_count,0.0,0.0,0.0,0.0,0.0
retweet_count,0.0,0.0,0.0,0.0,0.0


In [11]:
# Descriptive statistics on likes and retweets
df[['like_count', 'retweet_count']].describe()

,like_count,retweet_count
count,499974.000000,499974.000000
mean,7.123208,1.481523
std,216.366469,46.254101
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,0.000000
max,64094.000000,16080.000000
